<div style="background-color: steelblue; padding: 10px; border-radius: 5px;">
    <p style="margin: 10;"></p>
    <h1 style="text-align: center; margin: 0; font-weight: bold; color: white;">SWOT Ocean Model Intercomparison Project : EORCA12-ERA5v1 Test Case</h1>
    <p style="margin: 10;"></p>
</div>



## 📦**Imports** 

In [1]:
%%time

##################################
#Imports

from datetime import datetime
import s3fs
import xarray as xr
import pyinterp
from widetrax import DataPreprocessing as dp
from widetrax import Spectra as sp
import numpy as np
import json
from watermark import watermark
import time
import platform

CPU times: user 1.71 s, sys: 2.65 s, total: 4.36 s
Wall time: 1.73 s


## 🔍 **Check the S3 Endpoint**  

In [2]:
s3_folder ="s3://npd-eorca025-era5v1-swot-omip"
endpoint_url =  "https://noc-msm-o.s3-ext.jc.rl.ac.uk"
fs = s3fs.S3FileSystem(anon=True,endpoint_url=endpoint_url)
# List contents of the bucket
bucket_name = 's3://npd-eorca025-era5v1-swot-omip'
contents = fs.ls(bucket_name)
print("Bucket contents:")
for item in contents:
    print(item)

Bucket contents:
npd-eorca025-era5v1-swot-omip/cycle_008
npd-eorca025-era5v1-swot-omip/cycle_009
npd-eorca025-era5v1-swot-omip/cycle_010
npd-eorca025-era5v1-swot-omip/cycle_011
npd-eorca025-era5v1-swot-omip/cycle_012
npd-eorca025-era5v1-swot-omip/cycle_013
npd-eorca025-era5v1-swot-omip/cycle_014
npd-eorca025-era5v1-swot-omip/cycle_015
npd-eorca025-era5v1-swot-omip/cycle_016
npd-eorca025-era5v1-swot-omip/cycle_017
npd-eorca025-era5v1-swot-omip/cycle_018
npd-eorca025-era5v1-swot-omip/cycle_019
npd-eorca025-era5v1-swot-omip/cycle_020
npd-eorca025-era5v1-swot-omip/cycle_021
npd-eorca025-era5v1-swot-omip/cycle_022
npd-eorca025-era5v1-swot-omip/cycle_023
npd-eorca025-era5v1-swot-omip/cycle_024
npd-eorca025-era5v1-swot-omip/cycle_025
npd-eorca025-era5v1-swot-omip/cycle_026


## 🧮 **Required variables**

In [3]:
# REGION A

name_region="Region_B"
season="JFM"

if name_region=="Region_A":
    lon_min = 360-70,
    lon_max = 360-60,
    lat_min = 32,
    lat_max = 42,
elif name_region=="Region_B":
    lon_min = 360-50,
    lon_max = 360-40,
    lat_min = 17,
    lat_max = 27,

area = [lon_min, lat_min, lon_max, lat_max]

if season=="JFM":    
    start_date = "01012024" # "DDMMYYYY"
    end_date ="31032024"
elif season=="JAS":
    start_date = "01072024" # "DDMMYYYY"
    end_date ="30092024"

In [4]:
end_date

'30092024'

## 🔄 **Identify the cycle numbers within the specified time range** 

In [4]:
if season =="JFM":
    file_path = "https://minio.lab.dive.edito.eu/project-meom-ige/cycles_periods.csv" # works only for winter period
elif season =="JAS":
    file_path = "time_ranges.csv"  # for summer

matching_cycles = dp.get_matching_cycles(file_path, start_date, end_date)

def formater_numeros_concis(liste_numeros):
  return [str(numero).zfill(3) for numero in liste_numeros]
    
matching_cycles = formater_numeros_concis(matching_cycles)
matching_cycles


['008', '009', '010', '011', '012', '013']

## 📊 **Load datasets and compute Wavenumber spectra**

In [5]:
%%time

# Load datasets into a dictionary
datasets_dict = dp.read_swot_ncfiles_S3subfolders(
    s3_folder,
    matching_cycles,
    endpoint_url,
    area)

SSH_008 included.
SSH_008 included.
SSH_008 included.
SSH_008 included.
SSH_008 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_009 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_010 included.
SSH_011 included.
SSH_011 included.
SSH_011 included.
SSH_011 included.
SSH_011 included.
SSH_011 included.
SSH_011 included.
SSH_011 included.
SSH_011 in

In [6]:
%%time 
has_converged, filled_datasets = dp.fill_nan(datasets_dict, varname = "ssh")

CPU times: user 2.42 s, sys: 5.02 s, total: 7.44 s
Wall time: 3.2 s


In [7]:
%%time
segments_dict = sp.retrieve_segments(filled_datasets,FileType = "NetCDF",namevar="ssh")

/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/widetrax/Spectra.py:43: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  for col in range(dataset.dims['num_pixels']):


CPU times: user 276 ms, sys: 4.07 ms, total: 280 ms
Wall time: 278 ms


In [8]:
%%time
psd_dict, freqs_dict = sp.calculate_psd(segments_dict)
# Calculate PSD Mean
psd_mean, freqs_mean = sp.psd_mean_and_freq(psd_dict,freqs_dict)

CPU times: user 657 ms, sys: 476 ms, total: 1.13 s
Wall time: 4.53 s


In [9]:
psd_mean

array([2.90501928e-02, 1.28784503e-01, 1.12117076e-01, 7.40979251e-02,
       2.50792340e-02, 6.38721167e-03, 2.44570150e-03, 1.36745831e-03,
       6.01374193e-04, 2.91431306e-04, 1.33074761e-04, 7.29309005e-05,
       3.14586169e-05, 1.68042002e-05, 8.82484098e-06, 4.94108518e-06,
       3.04940899e-06, 1.90974759e-06, 2.07088305e-06, 1.55625287e-06,
       1.29563829e-06, 1.92973695e-06, 2.00487551e-06, 1.16486339e-06,
       1.59570583e-06, 1.83668933e-06, 1.83205688e-06, 2.25992575e-06,
       2.31818711e-06, 2.21114654e-06, 2.59672486e-06, 3.00650324e-06,
       3.56530186e-06, 5.41246377e-06, 7.98277479e-06, 9.15481814e-06,
       9.24248220e-06, 9.68431596e-06, 1.07775185e-05, 1.46954055e-05,
       2.49890291e-05, 3.37316966e-05, 2.83825342e-05, 1.48478872e-05,
       5.89269119e-06, 5.58743870e-06, 1.25532659e-05, 1.81020185e-05,
       1.21491532e-05, 8.18283619e-06, 6.00349119e-06, 3.38448200e-06,
       2.08797227e-06, 1.74886394e-06, 1.82890847e-06, 1.47493781e-06,
      

## 💾 **Save Results and Information in JSON File**

In [10]:
# Infos
nom_region = name_region
model = "NOC-eORCA025"
name_season = season
freqs_mean = np.array(freqs_mean)
psd_mean = np.array(psd_mean)
todays_Date = datetime.now()
date_creation = todays_Date.isoformat() #ISO 8601 format
packages_versions = watermark(packages="numpy,widetrax,xarray,s3fs,pyinterp,datetime,json,platform,time")
machine_info = {
    "system": platform.system(),
    "node_name": platform.node(),
    "release": platform.release(),
    "version": platform.version(),
    "machine": platform.machine(),
    "processor": platform.processor(),
    "python_version": platform.python_version()
}

# JSON file structure
donnees = {
    "region": nom_region,
    "model": model,
    "lon_min": lon_min,
    "lon_max": lon_max,
    "lat_min": lat_min,
    "lat_max": lat_max,
    "start_date": datetime.strptime(start_date, "%d%m%Y").strftime("%Y-%m-%d"),
    "end_date": datetime.strptime(end_date, "%d%m%Y").strftime("%Y-%m-%d"),
    "packages_versions": packages_versions,
    "date_creation": date_creation,
    "login_creator": platform.node(),
    "Frequency": freqs_mean.tolist(),
    "Spectra": psd_mean.tolist(),
    "machine_info": machine_info
}

# Save on JSON file
with open(str(nom_region)+"_"+str(name_season)+"_"+str(model)+".json", "w", encoding="utf-8") as f:
    json.dump(donnees, f, ensure_ascii=False, indent=4)
#with open("OMIP_"+str(model)+"_"+str(nom_region)+"_"+str(name_season)+".json", "w", encoding="utf-8") as f:
  #  json.dump(donnees, f, ensure_ascii=False, indent=4)

#<region>-<period>-<data-source>.json
print("JSON file created")

JSON file created


## 📤 **Export Results to the S3 Endpoint** 

In [11]:
fs = s3fs.S3FileSystem( anon=True, endpoint_url="https://minio.lab.dive.edito.eu", use_ssl=False ) 

In [12]:
json_file = str(nom_region)+"_"+str(name_season)+"_"+str(model)+".json"
fs.put(json_file, "project-meom-ige/OMIP/")

[None]

In [16]:
fs